In [ ]:
import logging
import os.path

import numpy as np
import pymc as pm
import aesara.tensor as at
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import scipy
from scipy import stats
import pandas as pd
import warnings
import extract_correct_csv

from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

scaler = StandardScaler()

prova_3_subj = extract_correct_csv.extract_only_valid_subject()
valid_k_list = list(range(1, 10))

k=5
i =2
global_e_labels = []
global_subject = []
num_trials_to_remove = 48

for i in prova_3_subj:
    subj_ = extract_correct_csv.read_correct_subject_csv(i)
    csv_ = 'data/LookAtMe_0'+str(subj_)+'.csv'
    global_data = pd.read_csv(csv_, sep='\t')
    y = np.array(list([int(d>2) for d in global_data['rating']]))
    e_labels = y[:,np.newaxis]  # rating > 2
    e_labels = e_labels[num_trials_to_remove:]
    global_e_labels = global_e_labels + e_labels.tolist()
    subject = np.array(list([s for s in global_data['subject']]))[:, np.newaxis]
    subject = subject[num_trials_to_remove:]
    global_subject = global_subject + subject.tolist()

#e_labels = e_labels[num_trials_to_remove:]
'''N_e = e_labels.shape[0]
D_e = e_labels.shape[1]'''
global_e_labels = np.array(global_e_labels)
global_subject = np.array(global_subject)

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
global_e_labels

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [ ]:

N_e = global_e_labels.shape[0]
D_e = global_e_labels.shape[1]

N_sub = global_subject.shape[0]
D_sub = global_subject.shape[1]

global_subject_df = pd.DataFrame(global_subject, columns=['subject'])

subject_dict = dict(zip(global_subject_df.subject.unique(), range(len(prova_3_subj))))
subj_def = global_subject_df.replace(subject_dict).values

NUM_TRIAL = 160
TRIAL = NUM_TRIAL*len(prova_3_subj)
def populate_array(x, name):
    return name[NUM_TRIAL*(x-1)+num_trials_to_remove:NUM_TRIAL*x]

hr_temp = np.concatenate([pd.read_csv('data/features/hr/'+str(x)+'.csv') for x in prova_3_subj])
hr = np.concatenate([populate_array(x, hr_temp) for x in range(1, len(prova_3_subj)+1)])

pupil_temp = np.concatenate([pd.read_csv('data/features/pupil/'+str(x)+'.csv') for x in prova_3_subj])
pupil = np.concatenate([populate_array(x, pupil_temp) for x in range(1, len(prova_3_subj)+1)])

eda_temp = np.concatenate([pd.read_csv('data/features/eda/'+str(x)+'.csv') for x in prova_3_subj])
eda = np.concatenate([populate_array(x, eda_temp) for x in range(1, len(prova_3_subj)+1)])

N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = k

In [ ]:
print(N_hr,D_hr)
print(N_eda,D_eda)
print(N_pupil,D_pupil)
print(K)

1904 60
1904 60
1904 5
5


In [ ]:
with pm.Model() as sPPCA:
    # dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    pupil_data = pm.MutableData("pupil_data", pupil.T)
    eda_data = pm.MutableData("eda_data", eda.T)

    e_data = pm.ConstantData("e_data", global_e_labels.T)

    # matrici pesi
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]),
                       shape=[D_pupil, K])

    Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    # weight matrix for pain expectation.
    # check mu,sigma,shape
    We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e, K]), shape=[D_e, K])

    # latent space
    c = pm.Normal('c', mu=at.zeros([N_hr, K]), sigma=at.ones([N_hr, K]), shape=[N_hr, K])

    # dati dell'hrv interpretati come una gaussiana
    mu_hr = pm.Normal('mu_hr', Whr.dot(c.T), at.ones([D_hr, N_hr]))  # hyperprior 1
    sigma_hr = pm.Exponential('sigma_hr', at.ones([D_hr, N_hr]))  # hyperprior 2
    x_hr = pm.Normal('x_hr', mu=mu_hr, sigma=sigma_hr, shape=[D_hr, N_hr], observed=hr_data)

    # dati della dilatazione pupille interpretati come una gaussiana
    mu_pupil = pm.Normal('mu_pupil', Wpupil.dot(c.T), at.ones([D_pupil, N_pupil]))  # hyperprior 1
    sigma_pupil = pm.Exponential('sigma_pupil', at.ones([D_pupil, N_pupil]))  # hyperprior 2
    x_pupil = pm.Normal('x_pupil', mu=mu_pupil, sigma=sigma_pupil, shape=[D_pupil, N_pupil],
                        observed=pupil_data)

    # eda
    mu_eda = pm.Normal('mu_eda', Weda.dot(c.T), at.ones([D_eda, N_eda]))  # hyperprior 1
    sigma_eda = pm.Exponential('sigma_eda', at.ones([D_eda, N_eda]))  # hyperprior 2
    x_eda = pm.Normal('x_eda', mu=mu_eda, sigma=sigma_eda, shape=[D_eda, N_eda], observed=eda_data)

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    x_e = pm.Bernoulli('x_e', p=pm.math.sigmoid(We.dot(c.T)), shape=[D_e, N_e], observed=e_data)

    # x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    # x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [ ]:

gv = pm.model_to_graphviz(sPPCA)
gv.view('complete_pooling')

'complete_pooling.pdf'

In [ ]:
with sPPCA:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

Interrupted at 268 [0%]: Average Loss = 3.7685e+11


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import arviz as az

In [ ]:
az.plot_trace(trace)

In [ ]:
az.rhat(trace)

In [ ]:
fig, ax = plt.subplots()

max_rhat = (az.rhat(trace)
               .max()
               .to_array())
nvar, = max_rhat.shape

ax.barh(np.arange(nvar), max_rhat);
ax.axvline(1, c='k', ls='--', label="Convergence");

#ax.set_xlim(left=0.8);
ax.set_xlabel(r"$\hat{R}$");

#ax.set_yticks(np.arange(nvar));
#ax.set_yticklabels(max_rhat.coords["variable"].to_numpy()[::-1]);

ax.legend();

In [ ]:
# az.plot_trace(trace);
with sPPCA:
    # update values of predictors:
    # pm.set_data({"pupil_data": pupil, "hr_data": hr, "eda_data": eda})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
eda_pred = posterior_predictive.posterior_predictive["x_eda"]

In [ ]:
eda_pred = np.squeeze(eda_pred.mean('draw', keepdims='false')[0]).to_numpy()

In [ ]:
edapred_ = eda_pred.T

eda_ = eda.to_numpy()

corrlist = []
for i in range(112):
    res = np.corrcoef(eda_[i], edapred_[i])[0][1]
    corrlist.append(res)
    print('trial '+str(i)+ ' corr: '+str(res.round(3)))

In [ ]:
def ccc(x, y):
    ''' Concordance Correlation Coefficient'''
    sxy = np.sum((x - x.mean()) * (y - y.mean())) / x.shape[0]
    rhoc = 2 * sxy / (np.var(x) + np.var(y) + (x.mean() - y.mean()) ** 2)
    return rhoc

In [ ]:
ccc(edapred_[0],eda_[0])

In [ ]:
eda_[0]

In [ ]:
edapred_[0].shape

In [ ]:
x=eda_[0]
y=eda_pred[0]

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
x- x.mean()

In [ ]:
y-y.mean()

In [ ]:
(x - x.mean()) * (y - y.mean())


In [ ]:
sxy= np.sum() / x.shape[0]

In [ ]:
conc = ccc(eda_[0], eda_pred[0])

In [ ]:
pearson_list = []
concord_list = []
for i in range(112):
    pear = np.corrcoef(eda_[i], edapred_[i])[0][1]
    conc = ccc(eda_[i], eda_pred[i])
    pearson_list.append(pear)
    concord_list.append(conc)
    # print('trial ' + str(i) + ' corr: ' + str(res.round(3)))

In [ ]:
mean_pear = round(np.mean(pearson_list), 4)
mean_corc = round(np.mean(concord_list), 4)

In [ ]:
mean_subj = round(np.mean(corrlist),4)


        logging.basicConfig(level=logging.INFO, filename="logfile", filemode="a+",
                            format="%(asctime)-15s %(levelname)-8s %(message)s")
        logging.info("Mean corr coeff eda-hr using subj: " + str(subj_) + " " + str(round(mean_subj, 2)) + " script: " +
             os.path.basename(__file__) + "latent space dims: " + str(K))
